# Aprendizagem supervisionada: Classificação

A aprendizagem supervisionada é um ramo da aprendizagem automática que se foca na construção de modelos capazes de prever ou classificar dados com base em exemplos etiquetados. A aprendizagem supervisionada abrange duas abordagens principais: classificação, usada para prever classes ou categorias discretas, e regressão, usada para prever valores contínuos. Tal como no caso da aprendizagem não supervisionada, em Python, a biblioteca [scikit-learn](https://scikit-learn.org/) é amplamente utilizada para implementar algoritmos de aprendizagem supervisionada e explorar técnicas de classificação, regressão e outras tarefas relacionadas. Este tutorial foca-se na aplicação e avaliação de diferentes algoritmos de classificação disponibilizados pela biblioteca *scikit-learn*. 


In [ ]:
import sklearn

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme()

Tal como para a aprendizagem não supervisionada, vamos usar o conjunto de dados [Iris](https://archive.ics.uci.edu/dataset/53/iris) como exemplo para a aplicação de abordagens de classificação:

In [ ]:
iris = sns.load_dataset('iris')
iris.sample(5)

## Preparação dos dados

A classificação tem como objetivo prever o valor de um determinado atributo discreto (*target*). Como tal, temos de definir qual é o atributo que queremos prever. No caso do conjunto de dados *Iris*, o objetivo é identificar a espécie de planta, tendo como base as características das suas flores. Por isso, vamos separar o atributo `species` dos restantes:

In [ ]:
y = iris['species'].astype('category')
X = iris.drop(columns=['species'])

Para estimar o desempenho de modelos de classificação e avaliar a sua capacidade de generalização para dados que não foram vistos durante o treino, é necessário um conjunto de dados de teste que é colocado de parte durante a construção dos modelos.

O conjunto de dados *Iris* não tem uma partição predefinida. Como tal, vamos usar a função `train_test_split` da biblioteca *scikit-learn* para particionar o conjunto. Neste caso, vamos usar 80% para treino e 20% para teste:

In [1]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Nota**: O argumento `random_state` é usado para que o resultado seja reprodutível.

Para visualizar os dois conjuntos, vamos usar PCA para obter os dois componentes principais:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X_train)

pca_train = pd.DataFrame(pca.transform(X_train), index=X_train.index, columns=['PC1', 'PC2'])
sns.scatterplot(pca_train, x='PC1', y='PC2', hue=y_train)
plt.title('Training Set')
plt.show()

pca_test = pd.DataFrame(pca.transform(X_test), index=X_test.index, columns=['PC1', 'PC2'])
sns.scatterplot(pca_test, x='PC1', y='PC2', hue=y_test)
plt.title('Test Set')
plt.show()

**Nota**: Apesar de a decomposição estar a ser feita apenas para visualização, é boa prática que esta seja baseada apenas no conjunto de treino. Ambos os conjuntos podem depois ser transformados de acordo com essa decomposição.

## Método de avaliação

Para avaliar o desempenho de um modelo de classificação é necessário definir um conjunto de métricas relevantes para o problema que se está a abordar. A função `classification_report` da biblioteca *scikit-learn* recebe as classificações de um determinado conjunto de dados e as previsões de um classificador para o mesmo conjunto e produz um resumo com as métricas de avaliação mais comuns para tarefas de classificação: taxa de acerto, precisão, cobertura e F1-Score. Para além disso, dados os mesmos argumentos, a função `confusion_matrix` constrói a matriz de confusão entre as classificações reais e as previstas. Para simplificar, vamos definir uma função que combina estas duas e apresenta os resultados:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

def display_evaluation(labels, predictions, class_names):
    print(classification_report(labels, predictions, target_names=class_names))
    cm = confusion_matrix(labels, predictions, labels=class_names)
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names).plot()
    plt.show()

**Nota**: O argumento `output_dict=True` pode ser usado para que a função `classification_report` retorne os resultados na forma de um dicionário. Isto é útil para agregar os resultados de vários modelos e compará-los mais tarde.

## Treino

Agora que temos o conjunto de dados particionado e definimos uma forma de avaliar o desempenho dos modelos, podemos começar a explorar algumas das abordagens de classificação disponibilizadas pela biblioteca *scikit-learn* de forma a treinar um conjunto de modelos que nos pareça adequado para o problema. No entanto, se olharmos para o desempenho de um modelo no mesmo conjunto de dados em que este foi treinado, os resultados vão estar inflacionados e não refletem a sua capacidade de generalização. Isto pode levar à seleção de modelos que acabam por ter um desempenho muito pior no conjunto de teste. Como tal, é costume fazer uma outra partição do conjunto de treino de forma a usar parte deste para validação. Desta forma, enquanto as diversas abordagens de classificação estão a ser exploradas, os modelos são treinados num subconjunto dos dados treino e avaliados nesse subconjunto de validação. As abordagens (e parametrizações) com melhor desempenho no conjunto de validação são então selecionadas, sendo o conjunto de treino completo usado para treinar novos modelos de acordo com essas configurações. Desta forma, é maximizada a quantidade de dados de treino usada para gerar os modelos que serão depois avaliados no conjunto de teste.

Quando existem recursos (temporais/computacionais) suficientes, é possível fazer uma exploração mais completa, repetindo as experiências variando o conjunto de validação. A validação cruzada é uma técnica usada para fazer este tipo de exploração. Neste caso, o conjunto de treino é particionado em *k* subconjuntos e cada um deles é usado, à vez, como conjunto de validação enquanto os restantes são usados para treinar o modelo. A função `cross_val_score` da biblioteca *scikit-learn* aplica esta técnica automaticamente e devolve o valor da taxa de acerto para cada um dos *k* subconjuntos. Para calcular os valores de outras métricas, podemos usar a função `cross_val_predict` para obter as previsões para cada exemplo do conjunto de treino quando este faz parte do conjunto de validação.

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict

Como vamos explorar diferentes tipos de algoritmos de classificação, vamos criar um dicionário para guardar o melhor classificador de cada tipo para mais tarde avaliar o seu desempenho no conjunto de teste:

In [ ]:
classifiers = {}

Dada a quantidade de algoritmos de classificação disponibilizada pela biblioteca *scikit-learn*, não vamos explorar a fundo todas as variações possíveis. No entanto, serão indicados alguns argumentos que podem ser usados para parametrizar as abordagens. O código disponível neste notebook pode ser alterado para explorar diferentes valores para esses argumentos, de forma a obter os modelos com melhor desempenho. 

**Nota**: Como o processo de validação cruzada e muitos dos algoritmos de classificação incluem fatores não determinísticos, recomenda-se que sejam feitas múltiplas execuções para tomar decisões mais informadas.

### k-NN

O algoritmo k-NN (k-Nearest Neighbors) é uma técnica simples de aprendizagem supervisionada. Tal como o nome indica, usando este algoritmo, a classificação de uma nova observação é determinada pelas classificações dos seus *k* vizinhos mais próximos. Na biblioteca *scikit-learn*, este algoritmo é implementado pela classe `KNeighborsClassifier`, sendo o número de vizinhos a considerar definido pelo argumento `n_neighbors`:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_cls = KNeighborsClassifier(n_neighbors=3)

**Nota**: Por predefinição, a classificação é dada pela maioria das classificações dos vizinhos. O argumento `weights='distance'` pode ser usado para fazer uma ponderação com base na distância.

Vamos então usar a função `cross_val_predict` para aplicar uma técnica de validação cruzada e obter as previsões para todos os exemplos do conjunto de treino:

In [ ]:
knn_cv_pred = cross_val_predict(knn_cls, X_train, y_train, cv=5)

Podemos usar a projeção nos dois componentes principais para visualizar as previsões corretas e incorretas:

In [ ]:
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=knn_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=knn_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               );

E a função que definimos anteriormente para obter os resultados em termos das várias métricas e a matriz de confusão:

In [ ]:
display_evaluation(y_train, knn_cv_pred, y.cat.categories)

Após encontrarmos a melhor configuração para o algoritmo, podemos treinar o modelo no conjunto de treino completo:

In [ ]:
classifiers['kNN'] = knn_cls.fit(X_train, y_train)

### Naive Bayes

O algoritmo Naive Bayes baseia-se no teorema de Bayes e assume independência condicional entre os vários atributos. Na biblioteca *scikit-learn* é implementado pela classe `GaussianNB`:

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_cls = GaussianNB()

In [ ]:
nb_cv_pred = cross_val_predict(nb_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=nb_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=nb_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, nb_cv_pred, y.cat.categories)

In [ ]:
classifiers['Naive Bayes'] = nb_cls.fit(X_train, y_train)

### Árvores de Decisão

As árvores de decisão são uma técnica de aprendizagem supervisionada que se baseia na construção de uma estrutura em forma de árvore para representar decisões de acordo com os valores dos atributos e os seus resultados. A biblioteca *scikit-learn* implementa esta técnica na classe `DecisionTreeClassifier`:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_cls = DecisionTreeClassifier()

**Nota**: Por predefinição, é usada a [impureza de Gini](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) como critério de seleção do atributo a testar em cada nó da árvore. Este critério pode ser substituído pela entropia usando o argumento `criterion='entropy'`. 

Para além disso, existem vários argumentos que podem ser usados para limitar a construção da árvore para evitar um sobreajustamento aos dados de treino. Por exemplo:

- `max_depth`: Profundidade máxima da árvore
- `min_samples_split`: Mínimo de exemplos necessários para continuar a tomar decisões
- `max_features`: Máximo de atributos a testar

Recomenda-se a consulta da [documentação da classe](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) para obter mais informação sobre as possibilidades de configuração.

In [ ]:
dt_cv_pred = cross_val_predict(dt_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=dt_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=dt_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, dt_cv_pred, y.cat.categories)

In [ ]:
classifiers['Decision Tree'] = dt_cls.fit(X_train, y_train)

Após treinar um classificador baseado em árvores de decisão, a árvore pode ser visualizada usando a função `plot_tree`:

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plot_tree(dt_cls, feature_names=X_train.columns, class_names=dt_cls.classes_, impurity=False, rounded=True, filled=True, max_depth=2);

Este tipo de visualização é útil para identificar quais os atributos mais relevantes para a classificação.

#### Floresta Aleatória (Random Forest)

O algoritmo Random Forest combina as previsões de múltiplas árvores de decisão treinadas usando subconjuntos aleatórios do conjunto de treino e dos atributos. Na biblioteca *scikit-learn*, o algoritmo é implementado pela classe `RandomForestClassifier`, sendo o número de árvores de decisão definido pelo atributo `n_estimators`:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_cls = RandomForestClassifier(n_estimators=10)

**Nota**: As árvores de decisão podem ser parametrizadas usando os argumentos descritos anteriormente.

In [ ]:
rf_cv_pred = cross_val_predict(rf_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=rf_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=rf_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, rf_cv_pred, y.cat.categories)

In [ ]:
classifiers['Random Forest'] = rf_cls.fit(X_train, y_train)

### Regressão Logística

Apesar do nome, a regressão logística é maioritariamente usada para classificação. Este algoritmo usa uma função logística para modelar a probabilidade de uma classe em relação aos atributos. Na biblioteca *scikit-learn* é implementado pela classe `LogisticRegression`:

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_cls = LogisticRegression(max_iter=5000)

**Nota**: Como os parâmetros do modelo são calculados usando abordagens de otimização incremental, é possível que não seja alcançada a convergência. O argumento `max_iter` é usado para definir o número máximo de iterações. Para além disso, os argumentos `penalty` e `C` podem ser usados para parametrizar a abordagem de regularização de pesos usada pelo algoritmo e evitar o sobreajustamento.

In [ ]:
lr_cv_pred = cross_val_predict(lr_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=lr_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=lr_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, lr_cv_pred, y.cat.categories)

In [ ]:
classifiers['Logistic Regression'] = lr_cls.fit(X_train, y_train)

### Máquinas de Vetores de Suporte

As máquinas de vetores de suporte (SVMs) são uma técnica de aprendizagem supervisionada que se foca em encontrar um hiperplano de separação que maximiza a margem entre classes. Na biblioteca *scikit-learn*, esta técnica é implementada pela classe `SVC`:

In [ ]:
from sklearn.svm import SVC

#### Linear

Na sua versão original e mais simples, uma máquina de vetor de suporte é um classificador linear:

In [ ]:
linear_svm_cls = SVC(kernel='linear', C=1)

**Nota**: O argumento `C` é usado para controlar a suavidade da margem. 

In [ ]:
linear_svm_cv_pred = cross_val_predict(linear_svm_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=linear_svm_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=linear_svm_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, linear_svm_cv_pred, y.cat.categories)

In [ ]:
classifiers['Linear SVM'] = linear_svm_cls.fit(X_train, y_train)

#### Kernels

O truque do kernel é uma técnica usada no contexto das máquinas de vetores de suporte para que estas sejam capazes de lidar com conjuntos de dados não linearmente separáveis. A ideia por trás do truque do kernel é mapear os dados originais num espaço de maior dimensionalidade, onde é mais provável que esses dados sejam linearmente separáveis. Recomenda-se a consulta da [documentação da classe `SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) para obter informação sobre os vários tipos de kernel que podem ser usados. Como exemplo, vamos usar um kernel [RBF](https://en.wikipedia.org/wiki/Radial_basis_function):

In [ ]:
rbf_svm_cls = SVC(kernel='rbf')

In [ ]:
rbf_svm_cv_pred = cross_val_predict(rbf_svm_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=rbf_svm_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=rbf_svm_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, rbf_svm_cv_pred, y.cat.categories)

In [ ]:
classifiers['RBF SVM'] = rbf_svm_cls.fit(X_train, y_train)

### Redes Neuronais

As redes neuronais são modelos computacionais inspirados no cérebro humano. Elas consistem em neurónios conectados em camadas. Cada neurónio recebe entradas, realiza cálculos e produz uma saída. Durante o treino, os pesos associados às conexões entre os neurónios são ajustados para que a rede possa aprender padrões nos dados.

#### Perceptrão

O perceptrão é um classificador linear e é a versão mais simples de uma rede neuronal. No caso de um problema de classificação binária, consiste em apenas um neurónio. Em problemas multiclasse, há um neurónio por classe. Na biblioteca *scikit-learn* é implementado pela classe `Perceptron`:

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
p_cls = Perceptron()

**Nota**: Tal como no caso da regressão logística, o argumento `max_iter` pode ser usado para definir o número máximo de iterações. A regularização dos pesos pode ser controlada pelos argumentos `penalty` e `alpha`.

In [ ]:
p_cv_pred = cross_val_predict(p_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=p_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=p_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, p_cv_pred, y.cat.categories)

In [ ]:
classifiers['Perceptron'] = p_cls.fit(X_train, y_train)

#### Redes Multicamada

Redes neuronais com várias camadas podem ser usadas para aprender qualquer função e, como tal, lidar com conjuntos de dados que não são separáveis linearmente. Na biblioteca *scikit-learn*, a classe `MLPClassifier` implementa um perceptrão multicamada, sendo o número de camadas e neurónios por camada controlado pelo argumento `hidden_layer_sizes`. Como exemplo, vamos criar uma rede neuronal com uma camada escondida:

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp_1h_cls = MLPClassifier(hidden_layer_sizes=(16,), max_iter=5000)

**Nota**: As redes neuronais em geral e, especificamente, a classe `MLPClassifier` têm muitos hiperparâmetros configuráveis. Por exemplo:

- `activation`: A função de activação aplicada pelos neurónios
- `alpha`: Intensidade da regularização
- `learning_rate`/`learning_rate_init`: Taxa de aprendizagem

Recomenda-se a consulta da [documentação da classe](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) para obter informação sobre todas as possibilidades de configuração.

In [ ]:
mlp_1h_cv_pred = cross_val_predict(mlp_1h_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=mlp_1h_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=mlp_1h_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, mlp_1h_cv_pred, y.cat.categories)

In [ ]:
classifiers['MLP (1 hidden)'] = mlp_1h_cls.fit(X_train, y_train)

Quando criamos redes com várias camadas escondidas, o número de neurónios em cada camada pode variar:

In [ ]:
mlp_2h_cls = MLPClassifier(hidden_layer_sizes=(16,8), max_iter=5000)

In [ ]:
mlp_2h_cv_pred = cross_val_predict(mlp_2h_cls, X_train, y_train, cv=5)
sns.scatterplot(pca_train, x='PC1', y='PC2', 
                hue=mlp_2h_cv_pred==y_train, palette={True: 'green', False: 'red'}, 
                style=mlp_2h_cv_pred==y_train, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_train, mlp_2h_cv_pred, y.cat.categories)

In [ ]:
classifiers['MLP (2 hidden)'] = mlp_2h_cls.fit(X_train, y_train)

## Teste

Após explorar várias abordagens no conjunto de treino, podemos selecionar as melhores e avaliar o seu desempenho no conjunto de teste. O método `score` dos classificadores treinados usando a biblioteca *scikit-learn* pode ser usado para obter a taxa de acerto desse classificador num determinado conjunto de dados. Vamos usar esse método para calcular a taxa de acerto dos vários classificadores que guardamos no conjunto de teste:

In [ ]:
scores = pd.Series({c_name: c.score(X_test, y_test) for c_name, c in classifiers.items()}, name='Accuracy')
scores

Para calcular outras métricas, podemos usar o método `predict` para obter as previsões de um classificador num determinado conjunto de dados. Por exemplo, podemos usar esse método para obter as previsões do perceptrão para conjunto de teste e usar a função de avaliação que definimos anteriormente para analisar as suas falhas:

In [ ]:
predictions = classifiers['Perceptron'].predict(X_test)
sns.scatterplot(pca_test, x='PC1', y='PC2', 
                hue=predictions==y_test, palette={True: 'green', False: 'red'}, 
                style=predictions==y_test, markers={True: 'P', False: 'X'},
                legend=False
               )
plt.show()
display_evaluation(y_test, predictions, y.cat.categories)